# Website scraper using scrappy

Will crawl a site and scrape documents. 
documents are saved in kb/web

To Do:
- [ ] Move this to a standalone Python application

In [ ]:
%pip install scrapy

In [ ]:

import subprocess, os, sys
import gradio as gr
from urllib.parse import urlparse, urljoin


# from urllib.parse import urljoin, urlparse
# from scrapy.crawler import CrawlerRunner
# from scrapy.utils.log import configure_logging
# from twisted.internet import reactor, defer
# import asyncio

with gr.Blocks() as demo:
    gr.Markdown("## Web Scraper")
    gr.Markdown("This is a simple web scraper that can be used to scrape investor relations pages.")
    
    url = gr.Textbox(label="Enter URL", placeholder="https://example.com")
    
    status = gr.Textbox(label="Status", interactive=False, value="Ready to scrape. Enter a URL and press Enter.", lines=5)

    def run_scraper(url):
        # Run the spider as a subprocess
        if not url.startswith("http"):
            url = "http://" + url
        # Extract the domain from the URL
        parsed_url = urlparse(url)
        domain = parsed_url.netloc.replace("www.", "")
        if not domain:
            return "Invalid URL. Please enter a valid URL."
        status.value = f"Scraping {url}..."
        # Run the spider using subprocess
        try:
            result = subprocess.run([sys.executable, 'spider_runner.py', url, domain], check=True, text=True, capture_output=True)
            status_value = f"Scraping completed for {url}."
            return result.stderr, status_value
        except subprocess.CalledProcessError as e:
            status_value = f"Error occurred: {e}"
            return f"Error: {e}", status_value
    
    output = gr.Textbox(label="Output", interactive=False)
    
    url.submit(run_scraper, inputs=url, outputs=[output,status]) 

demo.launch(inbrowser=True)

DEBUG: Resetting dropped connection: huggingface.co
DEBUG: connect_tcp.started host='api.gradio.app' port=443 local_address=None timeout=3 socket_options=None
DEBUG: connect_tcp.started host='127.0.0.1' port=7870 local_address=None timeout=None socket_options=None
DEBUG: connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1110ab790>
DEBUG: send_request_headers.started request=<Request [b'GET']>
DEBUG: send_request_headers.complete
DEBUG: send_request_body.started request=<Request [b'GET']>
DEBUG: send_request_body.complete
DEBUG: receive_response_headers.started request=<Request [b'GET']>
DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'date', b'Sun, 20 Jul 2025 09:21:40 GMT'), (b'server', b'uvicorn'), (b'content-length', b'4'), (b'content-type', b'application/json')])
INFO: HTTP Request: GET http://127.0.0.1:7870/gradio_api/startup-events "HTTP/1.1 200 OK"
DEBUG: receive_response_body.started request=<Request [b'GET']>
D

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


DEBUG: connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1142b8d10>
DEBUG: start_tls.started ssl_context=<ssl.SSLContext object at 0x11414ad50> server_hostname='api.gradio.app' timeout=3


DEBUG: https://huggingface.co:443 "HEAD /api/telemetry/gradio/launched HTTP/1.1" 200 0
DEBUG: start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1146fcd50>
DEBUG: send_request_headers.started request=<Request [b'GET']>
DEBUG: send_request_headers.complete
DEBUG: send_request_body.started request=<Request [b'GET']>
DEBUG: send_request_body.complete
DEBUG: receive_response_headers.started request=<Request [b'GET']>
DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sun, 20 Jul 2025 09:21:41 GMT'), (b'Content-Type', b'application/json'), (b'Content-Length', b'21'), (b'Connection', b'keep-alive'), (b'Server', b'nginx/1.18.0'), (b'Access-Control-Allow-Origin', b'*')])
INFO: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
DEBUG: receive_response_body.started request=<Request [b'GET']>
DEBUG: receive_response_body.complete
DEBUG: response_closed.started
DEBUG: response_closed.complete
DEBUG: close.st

Crawling in Jupyter notebooks tends to crash the kernel as the CrawlerProcess' asyncio loop interferes with the Jupyter notebook's.
Kudos to Claude for spotting this. I had to run the scraper as a subprocess in order to make it work. 

In a production environment it would run as a python script in a container.

In [ ]:
# For Jupyter notebooks - test the scraper

import subprocess
import sys

result = subprocess.run([
    sys.executable, 'spider_runner.py',
    'https://www.eurobank.gr/en/group/investor-relations',
    'eurobank.gr'
], capture_output=True, text=True)

print("STDOUT:")
print(result.stdout)
print("STDERR:")
print(result.stderr)
print(f"Return code: {result.returncode}")